## Визуализация аудио сигнала

In [ ]:
import numpy as np  # Импорт библиотеки для работы с массивами и математическими операциями
import matplotlib.pyplot as plt  # Импорт библиотеки для построения графиков
from scipy.io import wavfile  # Импорт функции для чтения WAV файлов из библиотеки SciPy

# Чтение WAV файла: функция возвращает частоту дискретизации и аудиосигнал
sampling_freq, signal = wavfile.read('dataset/random_sound.wav')

# Вывод информации о загруженном аудиосигнале
print('\nФорма сигнала:', signal.shape)  # Размерность массива (количество отсчетов, количество каналов)
print('Тип данных:', signal.dtype)  # Тип данных массива (обычно int16 для 16-битного аудио)
print('Продолжительность сигнала:', round(signal.shape[0] / float(sampling_freq), 2), 'в секундах')
# Вычисление продолжительности: количество отсчетов / частоту дискретизации

# Нормализация сигнала: приведение значений к диапазону [-1, 1] для 16-битного аудио
# 2^15 = 32768 (максимальное значение для 16-битного знакового целого)
signal = signal / np.power(2, 15)

# Ограничение сигнала первыми 50 отсчетами для визуализации небольшого фрагмента
signal = signal[:50]

# Создание временной оси в миллисекундах для построения графика
# np.arange(0, len(signal), 1) создает массив индексов от 0 до 49
# Каждый индекс умножается на 1000/частоту дискретизации для перевода в миллисекунды
time_axis = 1000 * np.arange(0, len(signal), 1) / float(sampling_freq)

# Построение графика аудиосигнала
plt.plot(time_axis, signal, color='black')  # Черный график амплитуды сигнала
plt.xlabel('Время (милисекунды)')  # Подпись оси X
plt.ylabel('Амплитуда')  # Подпись оси Y
plt.title('Входной аудио сигнал')  # Заголовок графика
plt.show()  # Отображение графика

## Генерирование аудиосигналов

In [ ]:
import numpy as np  # Импорт библиотеки для работы с массивами и математических операций
import matplotlib.pyplot as plt  # Импорт библиотеки для построения графиков
from scipy.io.wavfile import write  # Импорт функции записи WAV файлов из библиотеки SciPy

# Имя файла для сохранения сгенерированного аудио
output_file = 'dataset/generated_audio.wav'

# Параметры генерации аудиосигнала
duration = 4  # Продолжительность сигнала в секундах
sampling_freq = 44100  # Частота дискретизации (стандартная для аудио - 44.1 кГц)
tone_freq = 784  # Частота тона в Гц (нота С5 в музыкальной нотации)
min_val = -4 * np.pi  # Минимальное значение для временной оси (в радианах)
max_val = 4 * np.pi  # Максимальное значение для временной оси (в радианах)

# Создание временной оси (массив временных меток)
# np.linspace создает равномерно распределенные значения от min_val до max_val
# Количество точек = длительность (сек) × частота дискретизации (Гц)
t = np.linspace(min_val, max_val, duration * sampling_freq)

# Генерация чистого синусоидального сигнала с заданной частотой
# Формула: A * sin(2π * f * t), где A=1 (амплитуда), f - частота тона
signal = np.sin(2 * np.pi * tone_freq * t)

# Добавление случайного шума к сигналу для создания более реалистичного звука
# np.random.rand генерирует случайные числа от 0 до 1 в форме массива
# 0.5 - коэффициент, определяющий уровень шума относительно сигнала
noise = 0.5 * np.random.rand(duration * sampling_freq)
signal += noise  # Добавляем шум к исходному сигналу

# Подготовка сигнала для записи в 16-битный WAV файл
scaling_factor = np.power(2, 15) - 1  # Максимальное значение для 16-битного знакового целого (32767)
signal_normalized = signal / np.max(np.abs(signal))  # Нормализация к диапазону [-1, 1]
signal_scaled = np.int16(signal_normalized * scaling_factor)  # Преобразование к 16-битному формату

# Запись сгенерированного аудиосигнала в WAV файл
write(output_file, sampling_freq, signal_scaled)

# Для визуализации берем только первые 200 отсчетов сигнала
signal = signal[:200]

# Создание временной оси в миллисекундах для графика
# arange создает массив индексов от 0 до 199
# Каждый индекс преобразуется во время: (индекс / частота дискретизации) * 1000
time_axis = 1000 * np.arange(0, len(signal), 1) / float(sampling_freq)

# Построение графика сгенерированного аудиосигнала
plt.plot(time_axis, signal, color='black')  # Черная линия сигнала
plt.xlabel('Время (миллисекунды)')  # Подпись оси X
plt.ylabel('Амплитуда')  # Подпись оси Y
plt.title('Сгенерированный аудиосигнал')  # Заголовок графика
plt.show()  # Отображение графика

## Извлечение речевых признаков

In [ ]:
import numpy as np  # Импорт библиотеки для работы с массивами и математическими операциями
import matplotlib.pyplot as plt  # Импорт библиотеки для построения графиков
from scipy.io import wavfile  # Импорт функции для чтения WAV файлов
from python_speech_features import mfcc, logfbank  # Импорт функций для извлечения аудио-признаков

# Чтение WAV файла: функция возвращает частоту дискретизации и аудиосигнал
sampling_freq, signal = wavfile.read('dataset/random_sound1.wav')

# Ограничиваем сигнал первыми 10000 отсчетами для анализа (примерно 0.23 сек при 44.1 кГц)
# Это ускоряет обработку и визуализацию
signal = signal[:10000]

# Извлечение MFCC (Mel-frequency cepstral coefficients - мел-кепстральные коэффициенты)
# MFCC - популярные признаки для распознавания речи, моделирующие восприятие звука человеком
features_mfcc = mfcc(signal, sampling_freq)

# Вывод информации о MFCC признаках
print('\nMFCC:\nКоличество окон =', features_mfcc.shape[0])  # Количество временных окон
print('Длина каждого признака =', features_mfcc.shape[1])  # Количество MFCC коэффициентов (обычно 13)

# Транспонирование матрицы MFCC для визуализации (временная ось по горизонтали, коэффициенты по вертикали)
features_mfcc = features_mfcc.T
plt.matshow(features_mfcc)  # Построение тепловой карты MFCC
plt.title('MFCC')  # Заголовок графика

# Извлечение признаков фильтр-банка (log mel-filter bank energies)
# Логарифмическая энергия полос мел-фильтров - более простые признаки, чем MFCC
features_fb = logfbank(signal, sampling_freq)

# Вывод информации о признаках фильтр-банка
print('\nФильтр-банк:\nКоличество окон =', features_fb.shape[0])  # Количество временных окон
print('Длина каждого признака =', features_fb.shape[1])  # Количество фильтров в банке (обычно 26)

# Транспонирование матрицы фильтр-банка для визуализации
features_fb = features_fb.T
plt.matshow(features_fb)  # Построение тепловой карты фильтр-банка
plt.title('Фильтр-банк')  # Заголовок графика

plt.show()  # Отображение обоих графиков

## Преобразование аудиосигналов в частотные интервалы

In [ ]:
import numpy as np  # Импорт библиотеки для работы с массивами и математических операций
import matplotlib.pyplot as plt  # Импорт библиотеки для построения графиков
from scipy.io import wavfile  # Импорт функции для чтения WAV файлов

# Чтение WAV файла: функция возвращает частоту дискретизации и аудиосигнал
sampling_freq, signal = wavfile.read('dataset/spoken_word.wav')

# Нормализация сигнала: приведение значений к диапазону [-1, 1] для 16-битного аудио
# 2^15 = 32768 (максимальное значение для 16-битного знакового целого)
signal = signal / np.power(2, 15) 

# Определение длины сигнала (количество отсчетов)
len_signal = len(signal)

# Вычисление половины длины сигнала (округление вверх)
# Для быстрого преобразования Фурье достаточно рассматривать половину спектра
# (из-за симметрии для вещественных сигналов)
len_half = np.ceil((len_signal + 1) / 2.0).astype(int)

# Выполнение быстрого преобразования Фурье (FFT) для перехода из временной области в частотную
# Результат - комплексные числа, представляющие амплитуду и фазу частотных составляющих
freq_signal = np.fft.fft(signal)

# Извлечение амплитудного спектра (модуль комплексных чисел)
# Берём только первую половину (положительные частоты) и нормируем на длину сигнала
freq_signal = abs(freq_signal[0:len_half]) / len_signal

# Возведение в квадрат для получения мощности (спектральная плотность мощности)
# Мощность пропорциональна квадрату амплитуды
freq_signal **= 2

# Длина массива спектра мощности
len_fts = len(freq_signal)

# Корректировка мощности с учетом симметрии БПФ:
# Для нечетного числа точек удваиваем все значения, кроме первого (нулевой частоты)
# Для четного числа точек удваиваем все значения, кроме первого и последнего
# Это необходимо, так как мы взяли только половину спектра
if len_signal % 2:
    freq_signal[1:len_fts] *= 2
else:
    freq_signal[1:len_fts-1] *= 2

# Преобразование мощности в децибелы (логарифмическая шкала)
# 10 * log10(x) - стандартная формула для перевода мощности в дБ
signal_power = 10 * np.log10(freq_signal)

# Создание частотной оси в кГц для построения графика
# Каждая точка спектра соответствует определенной частоте:
# частота = (индекс * частота_дискретизации) / длина_сигнала
x_axis = np.arange(0, len_half, 1) * (sampling_freq / len_signal) / 1000.0

# Построение графика спектра мощности
plt.figure()
plt.plot(x_axis, signal_power, color='black')  # Черный график зависимости мощности от частоты
plt.xlabel('Частота (кГц)')  # Подпись оси X (частота в килогерцах)
plt.ylabel('Мощность сигнала (дБ)')  # Подпись оси Y (мощность в децибелах)
plt.show()  # Отображение графика

## Синтезирование звуков для генерации музыки

In [ ]:
import json  # Импорт модуля для работы с JSON файлами (чтение/запись)
import numpy as np  # Импорт библиотеки для работы с массивами и математическими операциями
import matplotlib.pyplot as plt  # Импорт библиотеки для построения графиков
from scipy.io.wavfile import write  # Импорт функции записи WAV файлов

def tone_synthesizer(freq, duration, amplitude=1.0, sampling_freq=44100):
    """
    Синтезатор тона: генерирует синусоидальный сигнал заданной частоты.
    
    Args:
        freq: Частота тона в Герцах (Гц)
        duration: Длительность тона в секундах
        amplitude: Амплитуда сигнала (по умолчанию 1.0)
        sampling_freq: Частота дискретизации (по умолчанию 44100 Гц)
        
    Returns:
        Аудиосигнал в формате 16-битного целого числа
    """
    # Вычисление количества отсчетов = длительность × частота дискретизации
    num_samples = int(duration * sampling_freq)
    
    # Создание временной оси от 0 до duration с равномерным шагом
    time_axis = np.linspace(0, duration, num_samples)

    # Генерация синусоидального сигнала: A × sin(2π × f × t)
    signal = amplitude * np.sin(2 * np.pi * freq * time_axis)

    # Преобразование сигнала в 16-битный целочисленный формат для записи в WAV
    return signal.astype(np.int16) 

if __name__=='__main__':
    # Имена выходных файлов для одиночного тона и последовательности тонов
    file_tone_single = 'dataset/generated_tone_single.wav'
    file_tone_sequence = 'dataset/generated_tone_sequence.wav'

    # Имя JSON файла с соответствием нот и частот
    mapping_file = 'dataset/tone_mapping.json'

    # Загрузка таблицы соответствия нот и частот из JSON файла
    try:
        with open(mapping_file, 'r') as f:
            tone_map = json.loads(f.read())  # Чтение и парсинг JSON
    except FileNotFoundError:
        # Если файл не найден, создаем стандартное соответствие нот и частот
        print(f"Файл {mapping_file} не найден. Создаю его...")
        # Частоты для нот первой октавы (в Герцах):
        tone_map = {
            'C': 261.63,  # До
            'D': 293.66,  # Ре
            'E': 329.63,  # Ми
            'F': 349.23,  # Фа
            'G': 392.00,  # Соль
            'A': 440.00,  # Ля
            'B': 493.88   # Си
        }
        # Сохранение таблицы в JSON файл для последующего использования
        with open(mapping_file, 'w') as f:
            json.dump(tone_map, f)
        print(f"Файл {mapping_file} создан.")

    # Параметры для генерации одиночного тона
    tone_name = 'F'  # Нота Фа
    duration = 3     # Длительность 3 секунды
    amplitude = 12000  # Амплитуда (максимальное значение для 16-битного аудио ≈ 32768)
    sampling_freq = 44100  # Стандартная частота дискретизации для аудио

    # Получение частоты для выбранной ноты из таблицы соответствия
    tone_freq = tone_map[tone_name]

    # Генерация одиночного тона с помощью функции-синтезатора
    synthesized_tone = tone_synthesizer(tone_freq, duration, amplitude, sampling_freq)

    # Запись одиночного тона в WAV файл
    write(file_tone_single, sampling_freq, synthesized_tone)

    # Создание последовательности тонов: список кортежей (нота, длительность)
    tone_sequence = [('G', 0.4), ('D', 0.5), ('F', 0.3), ('C', 0.6), ('A', 0.4)]

    # Инициализация пустого массива для результирующего сигнала
    signal = np.array([])
    
    # Генерация и объединение всех тонов из последовательности
    for item in tone_sequence:
        tone_name = item[0]  # Получение названия ноты
        freq = tone_map[tone_name]  # Получение частоты ноты из таблицы
        note_duration = item[1]  # Получение длительности ноты

        # Генерация тона для текущей ноты
        synthesized_tone = tone_synthesizer(freq, note_duration, amplitude, sampling_freq)

        # Добавление сгенерированного тона к общему сигналу
        signal = np.append(signal, synthesized_tone, axis=0)

    # Запись последовательности тонов в WAV файл
    write(file_tone_sequence, sampling_freq, signal)
    
    # Вывод информации о созданных файлах
    print(f"Создан файл: {file_tone_single}")
    print(f"Создан файл: {file_tone_sequence}")